# 1. Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd

import torch

from monai.networks.nets import AutoEncoder

from scipy.spatial.distance import pdist, squareform

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt

# 2. Define Global Variables

In [2]:
PATH_DATASET = 'exampleBiopsySessionDataset'
PATCH_SIZE = (9, 9, 3)

# 3. Extract Imaging Biomarkers

## 3.1. Import Patches

In [3]:
patch_dict = np.load(os.path.join(PATH_DATASET, 'patch_dict.npy'), allow_pickle = True)

## 3.2. Load the Auto-Encoder Model

In [4]:
model = AutoEncoder(spatial_dims = 2,
                    in_channels = PATCH_SIZE[2],
                    out_channels = PATCH_SIZE[2],
                    channels = (4,8),
                    inter_channels = (8,8),
                    strides = (2,2),
                    kernel_size = 5)

model.load_state_dict(torch.load('autoencoder_weights'))

<All keys matched successfully>

## 3.3. Retrieve Imaging Biomarkers from the Autoencoder's Embedding Layer

In [5]:
X = []
MD = []
for d in patch_dict:
    patch_array = d['patch_array']
    x = torch.as_tensor(patch_array)
    x = model.encode(x)
    x = model.intermediate(x)
    x = x.flatten()
    x = x.detach().cpu().numpy()
    
    X.append(x)
    MD.append({key:val for key,val in d.items() if not 'patch_array' in key})
    
X = np.array(X)
MD = pd.DataFrame(MD)
Y = MD['label_core'].values.copy()

# 4. Utilize Imaging Biomarkers for Pathology Prediction

## 4.1. Specify Experiment Configuration

In [6]:
SEED = 123

# Cross Validation Configuration
CV_SPLITS = 5
CV_POLICY = StratifiedKFold(n_splits = CV_SPLITS, random_state = SEED, shuffle = True)

# Feature Selection Configuration
PEARSON_CORRELATION_THRESHOLD = 0.7

# Random Forest Configuration
M = 300
MAX_DEPTH = 5

# Classification Task Configuration
TARGET = -1 # E.g.: Binary Classificaiton (Benign vs. Rest)

## 4.2. Define Utility Functions

In [7]:
def select_features_pearson_corr(X, threshold = 0.7):
    corr_matrix = pd.DataFrame(X).corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)).fillna(0)

    return (upper < threshold).all(axis = 1)

## 4.3. Run the Prediction Pipeline

In [8]:
results = []
for idx_fold, (idx_train, idx_test) in enumerate(CV_POLICY.split(X, Y)):
    
    # Cross-validation Instance Sampling
    X_train, X_test = X[idx_train].copy(), X[idx_test].copy()
    y_train, y_test = Y[idx_train].copy(), Y[idx_test].copy()
    MD_train, MD_test = MD.iloc[idx_train,:].copy(), MD.iloc[idx_test,:].copy()

    # Feature Selection
    selected_features = select_features_pearson_corr(X = X_train, threshold = PEARSON_CORRELATION_THRESHOLD)
    X_train = X_train[:,selected_features]
    X_test = X_test[:,selected_features]
    
    # Modeling
    estimator = RandomForestClassifier(n_estimators = M, max_depth = MAX_DEPTH, random_state = SEED)
    estimator.fit(X_train, y_train == TARGET)
    y_pred = estimator.predict_proba(X_test)
    y_pred = pd.DataFrame(y_pred, columns = ['p{}'.format(c) for c in range(y_pred.shape[1])])
    
    # Store Results
    results_fold = pd.DataFrame({
        'idx': idx_test,
        'y_true': y_test == TARGET,
        'type_biopsy': MD_test['type_biopsy'].values,
        'id_patch': MD_test['id_patch'].values,
        'id_core': MD_test['id_core'].values
    })
    
    results_fold = pd.concat([results_fold, y_pred], axis = 1)
    results.append(results_fold)
    
results = pd.concat(results, axis = 0)

## 4.4. Display Predictions for a Specific Biopsy Core

In [9]:
results_core_level = pd.DataFrame(results.groupby(by = 'id_core').apply(lambda x: np.percentile(x['p1'], 90)), columns = ['p1']).reset_index()
results_core_level = pd.merge(left = results_core_level, right = results[['id_core', 'y_true']].drop_duplicates().reset_index(drop=True), on = 'id_core', how = 'left')
results_core_level.iloc[0]

id_core        7460
p1         0.991333
y_true         True
Name: 0, dtype: object